# Variables of Interest Generation

In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import corpus
import pandas as pd
import requests
import time
import json

# word search function inspired by PyDictionary's word lookup function, but optimized for speed
def check_if_word_exists(word):
    words_dictionary = json.load(open('../data/words_dictionary.json'))
    if words_dictionary.get(word.lower()) is None:
        return False
    return True

df = pd.read_csv('../data/movies_cleaned.csv')

sia = SentimentIntensityAnalyzer()

# for getting the remaining time estimate
start_time = time.time()

for index, row in df.iterrows():
    print(f'Processing row {index} of {len(df)}')
    time_remaining = ((time.time() - start_time) / (index + 1)) * (len(df) - index)
    minutes_remaining = time_remaining // 60
    print(f'Estimated time remaining: {minutes_remaining} minutes, {time_remaining % 60} seconds')
    # generate polarity scores
    title = row['title']
    title_sentiment = sia.polarity_scores(title)
    df.at[index, 'title_sentiment'] = title_sentiment['compound']

    overview = str(row['overview'])
    overview_sentiment = sia.polarity_scores(overview)
    df.at[index, 'overview_sentiment'] = overview_sentiment['compound']

    # generate length features
    title_length = len(title)
    df.at[index, 'title_length'] = title_length
    
    num_words_in_title = len(title.split())
    df.at[index, 'num_words_in_title'] = num_words_in_title

    title_words = title.split()
    average_title_word_length = sum(len(word) for word in title_words) / len(title_words)
    df.at[index, 'average_title_word_length'] = average_title_word_length

    # check if there is a word in the title that does not exist in the dictionary
    # first strip out common title punctuation, numbers, and stopwords like "of, the, a, etc."
    title = ''.join([char for char in title if char.isalpha() or char.isspace()])
    title_words_not_in_dict = [word for word in title.split() if word.lower() not in corpus.stopwords.words("english")]
    title_words_not_in_dict = [word for word in title_words_not_in_dict if not check_if_word_exists(word)]
    print(title_words_not_in_dict)
    df.at[index, 'has_made_up_word'] = len(title_words_not_in_dict) > 0
    df.at[index, 'made_up_word_count'] = len(title_words_not_in_dict)

    
df.to_csv('../data/movies_with_vars.csv', index=False)
df.describe()

Processing row 0 of 7242
Estimated time remaining: 0.0 minutes, 19.67491865158081 seconds
['Godzilla']
Processing row 1 of 7242
Estimated time remaining: 30.0 minutes, 57.410629630088806 seconds
[]
Processing row 2 of 7242
Estimated time remaining: 33.0 minutes, 8.222163518269781 seconds
[]
Processing row 3 of 7242
Estimated time remaining: 29.0 minutes, 56.34311485290527 seconds
[]
Processing row 4 of 7242
Estimated time remaining: 28.0 minutes, 7.743713283538682 seconds
[]
Processing row 5 of 7242
Estimated time remaining: 25.0 minutes, 7.942129810651295 seconds
[]
Processing row 6 of 7242
Estimated time remaining: 24.0 minutes, 23.718858310154474 seconds
[]
Processing row 7 of 7242
Estimated time remaining: 22.0 minutes, 44.2594800889492 seconds
[]
Processing row 8 of 7242
Estimated time remaining: 22.0 minutes, 32.819777170816906 seconds
[]
Processing row 9 of 7242
Estimated time remaining: 22.0 minutes, 47.81740379333496 seconds
[]
Processing row 10 of 7242
Estimated time remainin